In [ ]:
from pds4_tools.reader.core import pds4_read
import numpy as np
# from scipy.fft import fft, ifft, fftfreq
import matplotlib.pyplot as plt
import prestitch as pre
import bgr
# import energy_eq as eng
# import gain
# import bandpass as bp
import os
import struct
from tqdm import tqdm
import time


'''
Retrieving files and sorting them
'''
n, m = 0, 109
files = [os.path.join('/Users/coltenrodriguez/Desktop/Senior_Thesis/RoPeR_LF', f) for f in os.listdir('/Users/coltenrodriguez/Desktop/Senior_Thesis/RoPeR_LF') if '.2CL' in f]
files = sorted(files, key=lambda f: int(f[-10:-6]))[n:m]
length = len(files)

sample_depth = {6000:15000, 4000:10000, 2000:5000, 1000:2500} # number of total pixels in either R/I data: nanoseconds penetration
observable_depth_ns = 1400



'''
Stitching multiple data files together and gathering some preprocessing data for each file
'''
phase = [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

sample_meta = None
for f, i, j in zip(files, range(0, length), tqdm(range(length+1))): 
    fdata = pds4_read(f, quiet=True)[0].data
    if len(set([int.from_bytes(d[6], 'big', signed=False) for d in fdata])) > 1:
        raise Exception("Trigger mode is not homogenously fixed_distance")
    sample_meta = fdata[0] # might be useless
    scan2d = np.array([d[9] for d in fdata])
    effective_length = 48000//4 # This is just taken from the .CL -- it should be the same for all input files of the same frequency type (effective data legnth/4 bytes per float)
    half_length = effective_length // 2
    real_part_scan2d = np.array(scan2d[:, :half_length]).T
    
    sample_depth_px, real_part_scan2d_depth_adj = pre.depth_correction(real_part_scan2d)
    real_part_scan2d_depth_self_adj = pre.remove_self_detection(real_part_scan2d_depth_adj)
    real_part_scan2d_depth_self_adj = pre.phase_correction(real_part_scan2d_depth_self_adj, phase[i])

    # Assume this is all interpreted
    yaw = np.array([d[23] for d in fdata])
    pos_x = np.array([d[19] for d in fdata])
    pos_y = np.array([d[20] for d in fdata])
    pos_z = np.array([d[21] for d in fdata])
    if i == 0:
        arr = real_part_scan2d_depth_self_adj
        yaws = yaw
        poss_x, poss_y, poss_z = pos_x, pos_y, pos_z
        trace_dist_m = np.sum(np.array([int.from_bytes(d[7], 'big', signed=False) for d in fdata])*10e-3)
    else:
        arr = np.concatenate((arr, real_part_scan2d_depth_self_adj), axis=1)
        yaws = np.concatenate((yaws, yaw))
        poss_x, poss_y, poss_z = np.concatenate((poss_x, pos_x)), np.concatenate((poss_y, pos_y)), np.concatenate((poss_z, pos_z))
        trace_dist_m+=np.sum(np.array([int.from_bytes(d[7], 'big', signed=False) for d in fdata])*10e-3)



# arr = bgr.time_lag(arr)
# # arr, arr_shape = bgr.remove_bg(arr)
# arr, arr_shape = bgr.subtract_window_mean(arr, 30)
# arr, dupes = bgr.dedupe(arr, arr_shape)
# arr = eng.lateral_energy_equalization(arr, 10)
# arr = gain.apply_gain(arr, 0.006)

# dt = 1.162e-10  # e.g., 0.1162 ns sampling interval = 2048/238ns = ~8pix/ns
# arr = np.array([bp.bandpass_filter(trace, dt) for trace in arr.T]).T


# # arr = bgr.eigen_background_removal(arr) # Optional, more comprehensive bg removal to minimize reflections

og = arr

In [ ]:
n_plots = 6
fig, ax = plt.subplots(n_plots, 1, figsize=(15,10))
step = int(arr.shape[1]/n_plots)
scans = [arr[60:200, n*step:(n+1)*step] for n in range(0,n_plots)]
trace_dist_iterator = np.array([0, 1, 2, 3, 4, 5, 6])*trace_dist_m/n_plots
print(trace_dist_iterator)

for i in range(0,len(scans)):
    scan = scans[i]
    d = int(trace_dist_iterator[i+1])-int(trace_dist_iterator[i])
    observable_depth = (6000/15000)*200
    print(d, observable_depth)
    ax[i].imshow(scan, extent=[int(trace_dist_iterator[i]), int(trace_dist_iterator[i+1]), observable_depth, 0], aspect=(observable_depth*2)/d, origin='upper', cmap='seismic')
